In [129]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [517]:
df=pd.read_csv('D:/Python/Github_ML/Machine-Learning/data/_preprocessed.csv', index_col=0)
df.head()

,user-id,user-age,user-gender,user-nationality,user-knowledge,user-loyalty,user-loan,user-income,user-savings,user-properties,...,user-dependents,user-pension,product-type,product-risk,product-term,product-yield,transaction-id,year,month,score
0,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,1,1,30,1,5000,2019,9,0.061877
1,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,2,2,10,2,5001,2019,10,0.179746
2,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,0,2,10,2,5002,2019,9,0.767918
3,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,0,0,20,0,5003,2017,12,0.606241
4,501,25,1,0,0,0,0,5284,51806,0,...,5,113300,0,0,1,0,5004,2019,10,0.829035


# Task 1

Подготовьте данные, проведите обучение моделей:

1. Линейная регрессия;
2. Регрессия дерева решений;
3. LASSO;
4. Гребневая регрессия;
5. Elastic Net регрессия.

Найдите реализации методов в sklearn, оставьте в нотбуке ссылки на документацию. Найдите наилучшие гиперпараметры. Оцените качество моделей: R2, Mean Square Error(MSE), Root Mean Square Error(RMSE),  mean absolute error (MAE). Свои действия снабжайте пояснениями.

In [518]:
df.drop(columns=['score']).corrwith(df['score'])

user-id              0.002696
user-age             0.017890
user-gender         -0.006855
user-nationality     0.000355
user-knowledge      -0.005503
user-loyalty        -0.018476
user-loan            0.014349
user-income          0.018721
user-savings         0.012658
user-properties      0.010081
user-riskAversion    0.018151
user-marital         0.001943
user-dependents     -0.003292
user-pension         0.014464
product-type         0.016353
product-risk         0.011767
product-term        -0.017461
product-yield        0.011767
transaction-id       0.002602
year                -0.037698
month               -0.005215
dtype: float64

In [519]:
from sklearn.preprocessing import StandardScaler

In [520]:
#Поменяем индексы в датафрейме для удобства стандартизации
df=df.reindex(columns=[
 'user-age',
 'user-income',
 'user-savings',
 'user-properties',                   
 'user-dependents',  
 'user-pension',                   
 'product-term',   
 'year',
 'month',
    
 #'user-gender',
 #'user-nationality',
 #'user-knowledge',
 #'user-loyalty',
 #'user-loan',
 #'user-riskAversion',
 #'user-marital',
 #'product-type',
 #'product-risk',
 #'product-yield',
 'score'])

In [521]:
from sklearn.preprocessing import scale
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [522]:
df_shuffled = shuffle(df, random_state=12)
X = df_shuffled.iloc[:, :-1].values
y = df_shuffled["score"]

In [523]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#### Стандартизация 

v=np.split(X_train, [9], axis=1)

w=np.split(X_test, [9], axis=1)

scaler = StandardScaler()
scaler.fit(v[0])


X_train=np.hstack((scaler.transform(v[0]), v[1]))

X_test=np.hstack((scaler.transform(w[0]), w[1]))

In [524]:
from sklearn.preprocessing import MinMaxScaler

In [525]:
scal=MinMaxScaler()
scal.fit(X_train)
X_train=scal.transform(X_train)
X_test=scal.transform(X_test)

In [526]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Подбор гиперпараметров для линейной регрессии

In [527]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {"eta0": [ .001, .003, .01, .03, .1, .3, 1, 3]} 
clf = SGDRegressor(tol=.0001) 
n_iter_search = 8 
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_dist, 
                                   n_iter=n_iter_search, 
                                   cv=3, 
                                   scoring='neg_mean_squared_error') 
random_search.fit(X_train, y_train) 

print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка MSE: {}".format(random_search.best_score_))

Лучшие параметры: {'eta0': 0.01}
Лучшая оценка MSE: -0.08387274722040712


##  1. Линейная регрессия

[Ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

In [528]:
clf = SGDRegressor(tol=.001, eta0=0.01) 
clf.fit(X_train, y_train)

SGDRegressor()

In [529]:
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)
from sklearn.metrics import r2_score
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08101354164170642
RMSE: 0.284628778660392
MAE:  0.23492459557860007
R2:  -0.0010846590396487166


In [531]:
[print(f'{elem[0]}||||{elem[1]}') for elem in zip(y_test, y_pred)];

0.5914037986449627||||0.5089476293902353
0.5359308676275593||||0.5250864648311226
0.549369743292146||||0.5009677218530022
0.0452057044870338||||0.5313039584211973
0.2368605099114993||||0.5251296613250958
0.2081211292835601||||0.4973592806678241
0.5978569197582049||||0.5178793408750505
0.0||||0.5453523869281034
0.8114330949318124||||0.5378251277753276
0.2425022467581924||||0.5205299731574251
0.7057014915892817||||0.5359266593137553
0.0548613283257671||||0.5432432910017433
0.8375464452533434||||0.53813541563681
0.0113689779534325||||0.5049561269062962
0.7596480481045587||||0.5335055607929424
0.611741370014733||||0.5337706387529051
0.7445945977930639||||0.5082174348405604
0.8964824432228444||||0.5296446231839899
0.0907448846143282||||0.5338833749376318
0.7152848322701549||||0.5307484655228237
0.7585641877112499||||0.5306713585591147
0.8012526966602435||||0.5367033311912797
0.7264839851775188||||0.5182020498028301
0.54418020077352||||0.553302511621776
0.5276403229018687||||0.52001152892512

0.7470497578375715||||0.5404825979257206
0.6362454305229437||||0.5539192552355076
0.8476386159826651||||0.546725031158977
0.694689679344903||||0.5122172870237001
0.1776443313461641||||0.5376339034849931
0.2078387646981421||||0.5291650153029753
0.8943933937189752||||0.5002228021364985
0.5480325228734962||||0.514334639521633
0.2363908639992886||||0.5187611991364157
0.7848290111163914||||0.5201214873341872
0.1897057432346729||||0.5264314282706003
0.6954777733593148||||0.5548678543109674
0.8160955728121794||||0.5339917271541951
0.1312607998706486||||0.54798214740466
0.2985623134089468||||0.5333736459442223
0.1521679840696273||||0.5373183227190413
0.8309552209472276||||0.5309842596510072
0.6757503776692435||||0.53311485116933
0.034022232977774||||0.5206980582549323
0.0378454974404378||||0.5222290678440903
0.673406921607261||||0.5018271811440823
0.7106954908855518||||0.5304492088873922
0.5972474887888434||||0.5279969529152833
0.0||||0.5200285697970531
0.817269019362791||||0.5649868302758487


#### Коэффицент детерминации  <0 , следовательно простое усреднение 'score' лучше, чем линейная регрессия. Регрессия часто неприменима к реальным данным из-за слишком ограниченных возможностей и ограниченной свободы маневра. Ее часто используют только в качестве базовой модели для оценки и сравнения с новыми подходами в исследованиях.

## 2. Регрессия дерева решений

In [532]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [533]:
y_pred = regressor.predict(X_test)

In [534]:
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.16797697080084836
RMSE: 0.40984993692917454
MAE:  0.3266215185278841
R2:  -1.0756921019992869


## 3. Lasso

In [535]:
from sklearn.linear_model import \
    Lasso, Ridge, LassoCV,LinearRegression

In [536]:
clf = LassoCV()
clf.fit(X_train, y_train)

LassoCV()

In [537]:
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08090837196673027
RMSE: 0.2844439698195943
MAE:  0.23645308630314266
R2:  0.00021492305590298244


## 4. Гребневая регрессия 

In [538]:
from sklearn.linear_model import RidgeCV

clf = RidgeCV()
clf.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [539]:
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08091411467276398
RMSE: 0.28445406425777076
MAE:  0.2364638290261023
R2:  0.00014396041440856155


## 5.Elastic Net регрессия

In [540]:
from sklearn.linear_model import ElasticNetCV
clf = ElasticNetCV()
clf.fit(X_train, y_train)

ElasticNetCV()

In [541]:
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)
print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08090770202440163
RMSE: 0.28444279218219193
MAE:  0.23645351534742814
R2:  0.00022320153594534986


In [542]:
class LinearRegression() :
      
    def __init__(self, learning_rate, iterations) :       
        self.learning_rate = learning_rate         
        self.iterations = iterations
          
    # Обучение модели              
    def fit(self, X, y) :                   
        self.m, self.n = X.shape
          
        # Инициализация весов          
        self.W = np.zeros(self.n)         
        self.b = 0          
        self.X = X          
        self.y = y
                     
        for i in range(self.iterations) :          
            self.update_weights()             
        return self
      
    def update_weights(self) :             
        y_pred = self.predict(self.X)         
        # подсчет градиентов        
        dW = - ( 2 * ( self.X.T ).dot( self.y - y_pred )  ) / self.m       
        db = - 2 * np.sum( self.y - y_pred ) / self.m    
        
        # пересчет весов      
        self.W = self.W - self.learning_rate * dW      
        self.b = self.b - self.learning_rate * db
          
        return self
      
    # Функция предсказания   
    def predict( self, X ) :
      
        return X.dot( self.W ) + self.b

In [543]:
model = LinearRegression(iterations = 1000, learning_rate = 0.01)
model.fit(X_train, y_train)
      
# Предсказания
  
y_pred = model.predict(X_test)
      
mse = mean_squared_error(y_test, y_pred) 
print("MSE: ",mse)
print("RMSE: {}".format(np.sqrt(mse)))
mae = mean_absolute_error(y_test, y_pred)
print("MAE: ",mae)

print("R2: ", r2_score(y_test, y_pred))

MSE:  0.08187879697625558
RMSE: 0.28614471334668334
MAE:  0.23955637760485224
R2:  -0.01177662267950863
